# CH9. 실전 프로그램 개발 - Photo 앱

* 웹 사이트에 사진 등록   
* 사진들을 앨범으로 그룹화하여 관리   
* 각 사진에 대한 정보를 등록하고 열람

## 1. 애플리케이션 설계하기

> **테이블 설계**   
> Album - Photo (1:N 관계)   
> 1. Album 테이블   
> * id : 자동 생성   
> * name : CharField(50)   
> * description : CharField(100), Blank   
>   
> 2. Photo 테이블   
> * id : 자동 생성   
> * album : ForeignKey   
> * title : CharField(50)   
> * image : TumbnailImageField   
> * description : TextField, Blank   
> * upload_dt : DateTimeField, auto_now_add   

> **URL 설계**   
> 1. /photo/ : AlbumLV(ListView), album_list.html   
> 2. /photo/album/ : AlbumLV(ListView), album_list.html   
> 3. /photo/album/99 : AlbumDV(DetailView), album_detail.html   
> 4. /photo/photo/99 : PhotoDV(DetailView), photo_detail.html   

> **작업 순서**   
> 1. 포토 앱 생성 (startapp)   
> 2. settings.py : 포토 앱 등록   
> 3. urls.py : URL 등록   
> 4. views.py : 뷰 로직 작성   
> 5. templates 디렉토리 : 템플릿 파일 작성

## 2. 뼈대 만들기   

1. 포토 앱 생성   

```
python manage.py startapp photo   
```

2. 포토 앱 등록

```
mysite/settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'bookmark.apps.BookmarkConfig',
    'blog.apps.BlogConfig',
    'taggit.apps.TaggitAppConfig',
    'taggit_templatetags2',
    # 추가
    'blogs.apps.PhotoConfig',
]
```

## 3. 모델 코딩하기

: album, photo 테이블 생성하기

1. 테이블 생성

```
photo/models.py

from django.db import models
from django.urls import reverse
# 커스텀 필드 임포트
from photo.fields import ThumbnailImageField

class Album(models.Model) :
    # id 자동 생성
    name = models.CharField(max_length = 50)
    description = models.CharField('One Line Description', max_length = 100, blank = True)

    # 객체 출력 시 정렬 기준 (오름차순)
    class Meta :
        ordering = ('name',)

    def __str__(self) :
        return self.name

    def get_absolute_url(self) :
        return reverse('photo:album_detail', args = (self.id,))


class Photo(models.Model) :
    # id 자동 생성
    album = models.ForeignKey(Album, on_delete = models.CASCADE)
    title = models.CharFields('TITLE', max_length = 50)
    image = models.ThumbnailImageField(upload_to = 'photo/%Y/%m')
    description = models.TextField('Photo description', blank = True)
    upload_dt = models.DateTimeField('Upload Date', auto_now_add = True)

    class Meta:
        ordering = ('title',)

    def __str__(self):
        return self.title

    def get_absolute_url(self):
        return reverse('photo:photo_detail', args=(self.id,))
```

2. 어드민에 등록

```
photo/admin.py

from django.contrib import admin
from photo.models import Album, Photo

# 관계로 묶인 테이블 함께 보여주기
# StackedInline : 세로로 보여주기 (<-> TabularInline)
class PhotoInline(admin.StackedInline) :
    # 보여줄 테이블
    model = Photo
    # 보여줄 갯수
    extra = 2

@admin.register(Album)
@admin.register(Photo)
    
class AlbumAdmin(admin.ModelAdmin) :
    inlines = (PhotoInline,)
    list_display = ('id', 'name', 'description')
    
class PhotoAdmin(admin.ModelAdmin) :
    list_display = ('id', 'title', 'upload_dt')
```

3. 필드 정의하기

```
photo/fields.py

import os
from PIL import Image
from django.db.models.fields import ImageField, ImageFieldFile

# 썸네일로 병줄 이미지 필드 정의
class ThumbnailImageFieldFile(ImageFieldFile) :
    # 썸네일 이름 정의 (url, 경로 생성 시 이용)
    def _add_thumb(s) :
        parts = s.split(".")
        parts.insert(-1, "thumb")
        if parts[-1].lower() not in ['jpeg', 'jpg'] :
            parts[-1] = 'jpg'
        return ".".join(parts)
    
    # thumb_path 생성
    @property
        def thumb_path(self) :
            return self._add_thumb(self.path)
    
    # thumb_url 생성
    @property
        def thumb_url(self) :
            return self._add_thumb(self.url)

    # 파일 저장
    def save(self, name, content, save = True) :
        super().save(name, content, save)

        # 이미지 오픈
        img = Image.open(self.path)
        # 이미지 사이즈 재정의
        size = (self.field.thumb_width, self.field.thumb_height)
        img.thumbnail(size)
        # 이미지와 동일한 크기의 하얀색 배경 제작
        background = Image.new('RGB', size, (255, 255, 255))
        box = (int((size[0] - img.size[0]) / 2), int((size[1] - img.size[1]) / 2))
        background.paste(img, box)
        # thumb_path에 최종 이미지 파일 저장
        background.save(self.thumb_path, 'JPEG')

    # 이미지 삭제
    def delete(self, save = True) :
        # 이미지 존재 시, 경로에서 삭제
        if os.path.exists(self.thumb_path) :
            os.remove(self.thumb_path)
        # 썸네일 이미지도 삭제
        super().delete(save)


class ThumbnailImageField(ImageField) :
    attr_class = ThumbnailImageFieldFile

    def __init__(self, verbose_name = Nonem thumb_width = 128, thumb_heigth = 128, **kwargs) :
        # 썸네일 이미지 속성 지정
        self.thumb_width, self.thumb_height = thumb_width, thumb_height
        super().__init__(verbose_name, **kwargs)
```

3. 데이터 베이스에 반영

```
python manage.py makemigrations photo
python manage.py migrate
```

## 4. URL 설계하기

**1. mysite/urls.py 수정**

```
mysite/urls.py

from django.contrib import admin
from django.urls import path, include
from django.conf.urls.static import static
from django.conf import settings
from mysite.views import HomeView
# from bookmark.views import BookmarkLV, BookmarkDV
#from django.views.generic import ListView, DetailView
#from bookmark.models import Bookmark

urlpatterns = [
    # 루트 URL (첫 페이지)
    path('', HomeView.as_view(), name = 'home'),
    path('admin/', admin.site.urls),
    path('bookmark/', include('bookmark.urls')),
    path('blog/', include('blog.urls')),
    path('photo/', include('photo.urls'))

    #path('bookmark/', BookmarkLV.as_view(), name = 'index'),
    #path('bookmark/<int:pk>', BookmarkDV.as_view(), name = 'detail'),
    #path('bookmark/', ListView.as_view(model = Bookmark), name = 'index'),
    #path('bookmark/<int:pk>', DetailView.as_view(model = Bookmark), name = 'detail'),
] + static(settings.MEDIA_URL, document_root = settings.MEDIA_ROOT)
```


**2. photo/urls.py 추가**

```
photo/urls.py

from django.urls import path
from photo import views

app_name = 'photo'

urlpatterns = [
    path('', views.AlbumLV.as_view(), name = 'index'),
    path('album', views.AlbumLV.as_view(), name = 'album_list'),
    path('album/<int:pk>/', views.AlbumDV.as_view(), name = 'album_detail'),
    path('photo/<int:pk>/', views.PhotoDV.as_view(), name = 'photo_detail'),
]
```

## 5. 뷰 코딩하기

: urls.py에서 선언한 AlbumLV, AlbumDV, PhotoDV 정의하기

```
photo/views.py

from django.shortcuts import render
from django.views.generic import ListView, DetailView
from photo.models import Album, Photo

class AlbumLV(ListView) :
    model = Album
    

class AlbumDV(DetailView) :
    model = Album
    

class PhotoDV(DetailView) :
    model = Photo
```

## 6. 템플릿 코딩하기

**1. album_list.html**

```
photo/templates/photo/album_list.html

<!DOCTYPE html>
<html>

<body>
      {% extends "base.html" %}

      {% block title %} Album List {% endblock %}

      {% block extra-style %}
      <style>
        .thumbnail {
            border : 3px solid #ccc;
        }
      </style>
      {% endblock %}

      {% block content %}

            {% for item in object_list %}
                  <div class = "mt-5">
                      <a class = "h2" href = "{% url 'photo:album_detail' item.id %}">
                        {{ item.name }}
                      </a> &emsp;
                      <span class = "font-italic h5"> {{ item.description }} </span>
                  </div>

                  <hr style = "margin : 0 0 20px 0;">

                  <div class = "row">
                      {% for photo in item.photo_set.all|slice:":5" %}
                      <div class = "ml-5">
                          <div class = "thumnail">
                              <a href = "{{ photo.get_absolute_url }}">
                                  <img src = "{{ photo.image.thumb_url }}" style = "width: 100%;">
                              </a>
                          </div>
                      </div>
                      {% endfor %}
                  </div>
            {% endfor %}

    {% endblock %}

</body>

</html>
```

**2. album_detail.html**

```
photo/templates/photo/album_detail.html

<!DOCTYPE html>
<html>

<body>

    {% extends "base.html" %}

    {% block title %} Album Detail {% endblock %}

    {% block extra-style %}
    <style>
      .thumbnail {
          border : 5px solid #ccc;
      }
    </style>
    {% endblock %}

    {% block content %}

          <div class = "mt-5">
              <span class = "h2">{{ object.name }}&emsp;</span>
              <span class = "h5 font-italic">{{ object.description }}</span>
          </div>

          <hr style = "margin : 0 0 20px 0;">

          <div class = "row">

            {% for photo in object.photo_set.all %}
            <div class = "col-md-3 mb-5">
              <div class = "thumbnail">
                <a href = "{{ photo.get_absolute_url }}">
                <img src = "{{ photo.image.thumb_url }}" style = "width : 100%;">
                </a>
            </div>

            <ul>
                <li class = "font-italic">{{ photo.title }}</li>
                <li class = "font-italic">{{ photo.upload_dt|date:"Y-m-d" }}</li>
            </ul>
            </div>
           {% endfor %}

          </div>

    {% endblock %}

</body>

</html>
```

**3. photo_detail.html**

```
photo/templates/photo/photo_detail.html

<!DOCTYPE html>
<html>

<body>

    {% extends "base.html" %}
    {% block title %} Photo Detail {% endblock %}

    {% block content %}

        <h2 class = "mt-5">{{ object.title }}</h2>

        <div class = "row">
            <div class = "col-md-9">
                <a href = "{{ object.image.url }}">
                    <img src = "{{ object.image.url }}" style = "width : 100%;">
                </a>
            </div>

            <ul class = "col-md-3 mt-3">
                <li class = "h5"> Photo Descripition </li>
                    {% if object.description %} <p> {{ object.description | linebreaks }} </p>
                    {% else %} <p> (blank) </p>
                    {% endif %}
                <li class = "h5"> Album Name </li>
                    <p class = "font-italic">
                        <a href = "{% url 'photo:album_detail' object.album.id %}"> {{ object.album.name }} </a>
                    </p>
            </ul>
        </div>

        {% endblock %}

</body>

</html>
```

**4. base.html 수정**

```
mysite/templates/base.html

# 메뉴 버튼 추가

          <li class="nav-item mx-1 btn btn-link">
            <!-- aria-* 속성 삭제 -->
            <a class="nav-link text-white" href="{% url 'bookmark:index' %}"> Bookmarks </a>
          </li>

          <li class = "nav-item mx-1 btn btn-link">
            <a class = "nav-link text-white" href = "{% url 'photo:index' %}"> Photo </a>
          </li>
```

## 확인하기

http://localhost:8000

*데이터 추가 후 확인*